In [1]:
import torch

In [2]:
tensor = torch.tensor([0.1, 0.2, 0.3])

In [3]:
X = tensor.reshape(1,3)

In [4]:
X

tensor([[0.1000, 0.2000, 0.3000]])

In [5]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(in_features=3, out_features=3, bias=True)
        self.fc2 = torch.nn.Linear(in_features=3, out_features=1, bias=True)
    
    def forward(self, x):
        print(x)
        print(x.shape)
        x = self.fc1(x)
        print(x)
        print(x.shape)
        x = self.fc2(x)
        print(x)
        print(x.shape)
        return x

In [6]:
model = Net()

In [7]:
par = []
for parameter in model.parameters():
    par.append(parameter)   

In [8]:
import torch
from torch.optim import Optimizer
class _RequiredParameter(object):
    """Singleton class representing a required parameter for an Optimizer."""
    def __repr__(self):
        return "<required parameter>"

required = _RequiredParameter()




class SGD(Optimizer):
    r"""Implements stochastic gradient descent (optionally with momentum).
    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf
    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.
        Considering the specific case of Momentum, the update can be written as
        .. math::
                  v = \rho * v + g \\
                  p = p - lr * v
        where p, g, v and :math:`\rho` denote the parameters, gradient,
        velocity, and momentum respectively.
        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form
        .. math::
             v = \rho * v + lr * g \\
             p = p - v
        The Nesterov version is analogously modified.
    """

    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                print('-'*15)
                print('Parameter before update:',p)
                print('Shape of parameter before update:',p.shape)
                print('Gradient:',d_p)
                print('Shape of the gradient:',d_p.shape)
                print('X:',X)
                print('Learning rate:',-group['lr'])
                check = p.clone()
                p.data.add_(-group['lr'], d_p)
                print('Parameter of the tensor after update:', p)
                print('Sanity check:', check +(-group['lr']) * d_p)
                print('-'*15)

        return loss

In [9]:
optimizer_detail = SGD(model.parameters(), lr=1)

In [10]:
# criterion = torch.nn.MSELoss()


In [11]:
output = model(X)

tensor([[0.1000, 0.2000, 0.3000]])
torch.Size([1, 3])
tensor([[ 0.2572,  0.1864, -0.2532]], grad_fn=<AddmmBackward>)
torch.Size([1, 3])
tensor([[-0.5089]], grad_fn=<AddmmBackward>)
torch.Size([1, 1])


In [12]:
loss = output - 1

In [13]:
# loss = criterion(1, output)

In [14]:
optimizer_detail.zero_grad()

In [15]:
loss.backward()

In [16]:
optimizer_detail.step()

---------------
Parameter before update: Parameter containing:
tensor([[ 0.4722, -0.1605, -0.5165],
        [-0.4895,  0.2991,  0.1742],
        [ 0.0957, -0.1254,  0.2131]], requires_grad=True)
Shape of parameter before update: torch.Size([3, 3])
Gradient: tensor([[-0.0472, -0.0944, -0.1416],
        [-0.0556, -0.1112, -0.1668],
        [-0.0381, -0.0761, -0.1142]])
Shape of the gradient: torch.Size([3, 3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[ 0.5194, -0.0661, -0.3749],
        [-0.4339,  0.4103,  0.3410],
        [ 0.1337, -0.0492,  0.3273]], requires_grad=True)
Sanity check: tensor([[ 0.5194, -0.0661, -0.3749],
        [-0.4339,  0.4103,  0.3410],
        [ 0.1337, -0.0492,  0.3273]], grad_fn=<AddBackward0>)
---------------
---------------
Parameter before update: Parameter containing:
tensor([ 0.3971,  0.1233, -0.3016], requires_grad=True)
Shape of parameter before update: torch.Size([3])
Gradi

In [17]:
gradient = torch.tensor([-0.4719, -0.5560, -0.3805]).reshape(1,3)

In [18]:
gradient.t().matmul(X)

tensor([[-0.0472, -0.0944, -0.1416],
        [-0.0556, -0.1112, -0.1668],
        [-0.0380, -0.0761, -0.1142]])

In [48]:
loss

tensor([[-4.6789]], grad_fn=<SubBackward0>)

# Calculations
Going trough this from down to up because we are backpropagating:

OUTPUT:

```
---------------
---------------
Parameter before update: Parameter containing:
tensor([[ 0.4722, -0.1605, -0.5165],
        [-0.4895,  0.2991,  0.1742],
        [ 0.0957, -0.1254,  0.2131]], requires_grad=True)
Shape of parameter before update: torch.Size([3, 3])
Gradient: tensor([[-0.0472, -0.0944, -0.1416],
        [-0.0556, -0.1112, -0.1668],
        [-0.0381, -0.0761, -0.1142]])
Shape of the gradient: torch.Size([3, 3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[ 0.5194, -0.0661, -0.3749],
        [-0.4339,  0.4103,  0.3410],
        [ 0.1337, -0.0492,  0.3273]], requires_grad=True)
Sanity check: tensor([[ 0.5194, -0.0661, -0.3749],
        [-0.4339,  0.4103,  0.3410],
        [ 0.1337, -0.0492,  0.3273]], grad_fn=<AddBackward0>)
---------------
---------------
Parameter before update: Parameter containing:
tensor([ 0.3971,  0.1233, -0.3016], requires_grad=True)
Shape of parameter before update: torch.Size([3])
Gradient: tensor([-0.4719, -0.5560, -0.3805])
Shape of the gradient: torch.Size([3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([0.8690, 0.6793, 0.0789], requires_grad=True)
Sanity check: tensor([0.8690, 0.6793, 0.0789], grad_fn=<AddBackward0>)
---------------
---------------
Parameter before update: Parameter containing:
tensor([[-0.4719, -0.5560, -0.3805]], requires_grad=True)
Shape of parameter before update: torch.Size([1, 3])
Gradient: tensor([[ 0.2572,  0.1864, -0.2532]])
Shape of the gradient: torch.Size([1, 3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[-0.7292, -0.7424, -0.1273]], requires_grad=True)
Sanity check: tensor([[-0.7292, -0.7424, -0.1273]], grad_fn=<AddBackward0>)
---------------
---------------
Parameter before update: Parameter containing:
tensor([-0.3802], requires_grad=True)
Shape of parameter before update: torch.Size([1])
Gradient: tensor([1.])
Shape of the gradient: torch.Size([1])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([-1.3802], requires_grad=True)
Sanity check: tensor([-1.3802], grad_fn=<AddBackward0>)
---------------
```

Let's first consider:

```
---------------
Parameter before update: Parameter containing:
tensor([-0.3802], requires_grad=True)
Shape of parameter before update: torch.Size([1])
Gradient: tensor([1.])
Shape of the gradient: torch.Size([1])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([-1.3802], requires_grad=True)
Sanity check: tensor([-1.3802], grad_fn=<AddBackward0>)
---------------
```

This is obviously bias of the last layer or in other words second layer. We are taking derivative of wx+b w.r.t. b which is 1 which is fully confirmed by:

`
Gradient: tensor([1.])
`

Move backward to:

```
---------------
Parameter before update: Parameter containing:
tensor([[-0.4719, -0.5560, -0.3805]], requires_grad=True)
Shape of parameter before update: torch.Size([1, 3])
Gradient: tensor([[ 0.2572,  0.1864, -0.2532]])
Shape of the gradient: torch.Size([1, 3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([[-0.7292, -0.7424, -0.1273]], requires_grad=True)
Sanity check: tensor([[-0.7292, -0.7424, -0.1273]], grad_fn=<AddBackward0>)
---------------
```

wx+b w.r.t. to to w is x and x is output from first layer.
INPUT:

```
model(X)
```

OUTPUT:

```
tensor([[0.1000, 0.2000, 0.3000]])
torch.Size([1, 3])
tensor([[ 0.2572,  0.1864, -0.2532]], grad_fn=<AddmmBackward>)
torch.Size([1, 3])
tensor([[-0.5089]], grad_fn=<AddmmBackward>)
torch.Size([1, 1])
```

We are particulary interested in:

```
tensor([[ 0.2572,  0.1864, -0.2532]], grad_fn=<AddmmBackward>)
```

which is equal to gradient of the tensor in 2nd layer or output from 1st layer:

```
Gradient: tensor([[ 0.2572,  0.1864, -0.2532]])
```

Now we are dealing with bias in the first layer. 

```
---------------
Parameter before update: Parameter containing:
tensor([ 0.3971,  0.1233, -0.3016], requires_grad=True)
Shape of parameter before update: torch.Size([3])
Gradient: tensor([-0.4719, -0.5560, -0.3805])
Shape of the gradient: torch.Size([3])
X: tensor([[0.1000, 0.2000, 0.3000]])
Learning rate: -1
Parameter of the tensor after update: Parameter containing:
tensor([0.8690, 0.6793, 0.0789], requires_grad=True)
Sanity check: tensor([0.8690, 0.6793, 0.0789], grad_fn=<AddBackward0>)
---------------
```

The derivative of wx+b w.r.t. b in the first layer is equal to 1. So we multiply by 1 derivative of the first layer by wx+b w.r.t. x. Pay attention to the x which is w.
So, weight of the second layer is derivative of wx+b w.r.t. x in the second layer. Threfore, we multiply 1 * weight from previous layer(not updated, also pay attention to it) and it's equal to gradient of the bias in the first layer.

```
Gradient: tensor([-0.4719, -0.5560, -0.3805])
```

It's gradient of the bias which is equal to weight from previous layer. Have a look:

```
Parameter before update: Parameter containing:
tensor([[-0.4719, -0.5560, -0.3805]], requires_grad=True)
```

And finally, first layer wx+b w.r.t. to w is equal x. We have to multiply by gradient from previous layer in other words by w.
```
---------------
Parameter before update: Parameter containing:
tensor([[-0.4719, -0.5560, -0.3805]], requires_grad=True)
```

INPUT:

```
gradient = torch.tensor([-0.4719, -0.5560, -0.3805]).reshape(1,3)
gradient.t().matmul(X)
```

OUTPUT:

```
tensor([[-0.0472, -0.0944, -0.1416],
        [-0.0556, -0.1112, -0.1668],
        [-0.0380, -0.0761, -0.1142]])
```

which fully confirmed with autograd calculations:

```
Gradient: tensor([[-0.0472, -0.0944, -0.1416],
        [-0.0556, -0.1112, -0.1668],
        [-0.0381, -0.0761, -0.1142]])

```


In [21]:
X.shape

torch.Size([1, 3])

In [49]:
par[0]

Parameter containing:
tensor([[-0.2185,  0.1295,  0.1802],
        [-0.5210, -0.4478, -0.3567],
        [-0.4256,  0.3492,  0.4206]], requires_grad=True)

In [19]:
layer1 = X.matmul(par[0].t()) + par[1]
print(layer1)

tensor([[-0.3222, -0.4164, -0.2208]], grad_fn=<AddBackward0>)


In [20]:
layer2 = layer1.matmul(par[2].t()) + par[3]
print(layer2)

tensor([[-1.5294]], grad_fn=<AddBackward0>)


In [21]:
model(X)

tensor([[-0.3222, -0.4164, -0.2208]], grad_fn=<AddmmBackward>)
tensor([[-1.5294]], grad_fn=<AddmmBackward>)


tensor([[-1.5294]], grad_fn=<AddmmBackward>)

In [40]:
par

[Parameter containing:
 tensor([[-0.2185,  0.1295,  0.1802],
         [-0.5210, -0.4478, -0.3567],
         [-0.4256,  0.3492,  0.4206]], requires_grad=True),
 Parameter containing:
 tensor([ 0.8732, -0.8030, -0.1054], requires_grad=True),
 Parameter containing:
 tensor([[-0.8627,  1.0195,  0.0056]], requires_grad=True),
 Parameter containing:
 tensor([-1.3930], requires_grad=True)]